In [1]:
import os

import numpy as np
import xarray as xr

In this notebook, we take the data saved in `preprocess.ipynb` and use it to compute the residual circulation. Seviour et al. (2012) use a log-pressure vertical coordinate
$$z \equiv -H \log \frac{p}{p_{\textrm{surf}}}$$
with scale height $H = 6800 \textrm{ m}$. (At least, I believe that's what they mean &mdash; their description of the vertical coordinate seems to contain some typos.) They also define the reference density profile
$$\rho = \exp \left(-\frac{z}{H}\right)$$
Below, as we load the data, we convert the dataset's vertical dimension to this log-pressure coordinate, since most of the differentiation we perform later will be with respect to $z$.

In [2]:
data_dir = '/glade/work/dconnell/brewson'
fnames = [f'{data_dir}/{x}' for x in os.listdir(data_dir)]

with xr.open_mfdataset(fnames, combine='by_coords') as ds:
    ds = ds.rename({'level' : 'z'})
    p = (100 * ds['z']).assign_attrs(units='Pa')
    lat = (np.pi * ds['latitude'] / 180).assign_attrs(units='radians_north')
    
    H, p_surf = 6800, p[-1]
    z = (-H * np.log(p / p_surf)).assign_attrs(units='meters')
    rho = np.exp(-z / H)
    
    p = p.assign_coords(z=z)
    lat = lat.assign_coords(latitude=lat)
    rho = rho.assign_coords(z=z)
    
    ds = ds.assign_coords(z=z, latitude=lat)
    u, v, w_cartesian = ds['u'], ds['v'], ds['w']
    T, h = ds['T'], ds['Phi'] / 9.8
    
    display(ds)

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, season: 2, year: 43, z: 37)
Coordinates:
  * season     (season) object 'DJF' 'JJA'
  * z          (z) float64 4.697e+04 4.226e+04 3.95e+04 ... 348.8 172.2 -0.0
  * latitude   (latitude) float64 1.571 1.566 1.562 ... -1.562 -1.566 -1.571
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * year       (year) int64 1979 1980 1981 1982 1983 ... 2018 2019 2020 2021
Data variables:
    u          (year, season, z, latitude, longitude) float64 dask.array<shape=(43, 2, 37, 721, 1440), chunksize=(1, 2, 37, 721, 1440)>
    v          (year, season, z, latitude, longitude) float64 dask.array<shape=(43, 2, 37, 721, 1440), chunksize=(1, 2, 37, 721, 1440)>
    w          (year, season, z, latitude, longitude) float64 dask.array<shape=(43, 2, 37, 721, 1440), chunksize=(1, 2, 37, 721, 1440)>
    T          (year, season, z, latitude, longitude) float64 dask.array<shape=(43, 2, 37, 721, 1440), chunksize=(1

However, the ERA5 vertical velocity is with respect to the Cartesian vertical coordinate, which I'll denote by $h$. By the chain rule we of course have
$$\frac{\mathrm{d} z}{\mathrm{d} t} = \frac{\mathrm{d} z}{\mathrm{d} h}\frac{\mathrm{d} h}{\mathrm{d} t}$$
where the left-hand side is the vertical velocity we want to analyze and the last factor on the right-hand side is the ERA5 vertical velocity. In the cell below, we compute the desired $w$ field.

In [3]:
dzdh = 1 / h.differentiate('z')
w = dzdh * w_cartesian

Moving forward, it will be useful to have a function that performs Reynolds decomposition with a zonal mean. This is easy with `xarray`, and we provide the following wrapper.

In [4]:
def decompose(data):
    data_bar = data.mean('longitude')
    data_prime = data - data_bar
    
    return data_bar, data_prime

Next, we compute the potential temperature
$$\theta = \left(\frac{p_{\textrm{surf}}}{p}\right)^{R / c_{\mathrm{p}}}T$$
as well as the buoyancy frequency $N^2 = \bar{\theta}_z$. Note that we take $R / c_{\mathrm{p}} = 0.286$.

In [5]:
theta = T * ((p_surf / p) ** 0.286)
theta_bar, theta_prime = decompose(theta)
N2 = theta_bar.differentiate('z')

We are now in a position to compute the components of the residual circulation according to
$$\bar{v}^\ast = \bar{v} - \frac{1}{\rho}\left(\frac{\rho\overline{v'\theta'}}{N^2}\right)_z$$
and
$$\bar{w}^\ast = \bar{w} + \frac{1}{a\cos\vartheta}\left(\cos\vartheta \frac{\overline{v'\theta'}}{N^2}\right)_{\vartheta}$$
where $\vartheta$ is the latitude and $a$ is the radius of the Earth.

In [6]:
v_bar, v_prime = decompose(v)
w_bar, _ = decompose(w)
theta_flux = (v_prime * theta_prime).mean('longitude')

v_star = v_bar - (rho * theta_flux / N2).differentiate('z') / rho
w_star = w_bar + (np.cos(lat) * theta_flux / N2).differentiate('latitude') / (6.37e6 * np.cos(lat))